In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/Codes_divers/PlantBiophysics-paper-Quarto/notebooks/performance`


In [2]:
### A Pluto.jl notebook ###
# v0.19.23

using Markdown
using InteractiveUtils





# PlantBiophysics.jl benchmark

The main objective of this notebook is to compare the computational times of `PlantBiophysics.jl` against the [plantecophys](https://github.com/RemkoDuursma/plantecophys) R package and the [LeafGasExchange.jl](https://github.com/cropbox/LeafGasExchange.jl) Julia package from the [Cropbox.jl](https://github.com/cropbox/Cropbox.jl) framework. The comparison follows three steps:
- create an N-large basis of random conditions.
- benchmark the computational time of the three packages via similar functions (_i.e._ photosynthesis-stomatal conductance-energy balance coupled model for C3 leaves): `energy_balance`, `photosynEB` and `simulate` with `ModelC3MD`.
- compare the results with plots and statistics.

This notebook does not perform the benchmark by itself for the obvious reason that it takes forever to run, and because there is an overhead cause by Pluto (benchmark and reactive are not a good mix). Instead, it shows the outputs of [a script from this repository](https://github.com/VEZY/PlantBiophysics-paper/blob/main/tutorials/Fig5_PlantBiophysics_performance_noPluto.jl) that implements the code shown here. If you want to perform the benchmark by yourself, you can run this script from the command line. The versions used for the above dependencies are available in the [Project.toml](https://github.com/VEZY/PlantBiophysics-paper/blob/main/tutorials/Project.toml) of the repository.


## Importing the dependencies:

###### Note
Make sure to have R installed on your computer first.

Loading the Julia packages:

```julia
begin
    using CairoMakie
	using BenchmarkTools
	using PlantBiophysics
    using Cropbox
    using LeafGasExchange
    using RCall
end
```
	



In [3]:
begin
    using Statistics
    using DataFrames
    using CSV
    using Random
    using PlantBiophysics
	using Downloads
end





## Parameters

### Benchmark parameters

You'll find below the main parameters of the benchmark. In a few words, each package runs a simulation for `N` different time-steps `microbenchmark_steps` times repeated `microbenchmark_evals` times. We make `N` different simulations because the simulation duration can vary depending on the inputs due to iterative computations in the code, *i.e.* different initial conditions can make the algorithms converge more or less rapidly.



In [4]:
begin
    Random.seed!(1) # Set random seed
    microbenchmark_steps = 100 # Number of times the microbenchmark is run
    microbenchmark_evals = 1 # N. times each sample is run to be sure of the output
    N = 100 # Number of timesteps simulated for each microbenchmark step
end



100



### Random input simulation dataset

We create possible ranges for input parameters. These ranges where chosen so all of the three packages don't return errors during computation (`plantecophys` has issues with low temperatures).

- Ta: air temperature ($°C$)
- Wind: wind speed ($m.s^{-1}$)
- P: ambient pressure ($kPa$)
- Rh: relative humidity (between 0 and 1)
- Ca: air CO₂ concentration ($ppm$)
- Jmax: potential rate of electron transport ($\mu mol_{CO2}.m^{-2}.s^{-1}$)
- Vmax: maximum rate of Rubisco activity ($\mu mol_{CO2}.m^{-2}.s^{-1}$)
- Rd: mitochondrial respiration in the light at reference temperature ($\mu mol_{CO2}.m^{-2}.s^{-1}$)
- TPU: triose phosphate utilization-limited photosynthesis rate ($\mu mol_{CO2}.m^{-2}.s^{-1}$)
- Rs: short-wave net radiation ($W.m^{-1}$)
- skyF: Sun-visible fraction of the leaf (between 0 and 1)
- d: characteristic length ($m$)
- g0: residual stomatal conductance ($mol_{CO2}.m^{-2}.s^{-1}$)
- g1: slope of the stomatal conductance relationship.



In [5]:
begin
    # Create the ranges of input parameters
    length_range = 10000
    Rs = range(10, 500, length=length_range)
    Ta = range(18, 40, length=length_range)
    Wind = range(0.5, 20, length=length_range)
    P = range(90, 101, length=length_range)
    Rh = range(0.1, 0.98, length=length_range)
    Ca = range(360, 900, length=length_range)
    skyF = range(0.0, 1.0, length=length_range)
    d = range(0.001, 0.5, length=length_range)
    Jmax = range(200.0, 300.0, length=length_range)
    Vmax = range(150.0, 250.0, length=length_range)
    Rd = range(0.3, 2.0, length=length_range)
    TPU = range(5.0, 20.0, length=length_range)
    g0 = range(0.001, 2.0, length=length_range)
    g1 = range(0.5, 15.0, length=length_range)
    vars = hcat([Ta, Wind, P, Rh, Ca, Jmax, Vmax, Rd, Rs, skyF, d, TPU, g0, g1])
    nothing
end



We then sample `N` conditions from the given ranges:


In [6]:
begin
    set = [rand.(vars) for i = 1:N]
    set = reshape(vcat(set...), (length(set[1]), length(set)))'
    name = [
        "T",
        "Wind",
        "P",
        "Rh",
        "Ca",
        "JMaxRef",
        "VcMaxRef",
        "RdRef",
        "Rs",
        "sky_fraction",
        "d",
        "TPURef",
        "g0",
        "g1",
    ]
    set = DataFrame(set, name)
    @. set[!, :vpd] = e_sat(set.T) - vapor_pressure(set.T, set.Rh)
    @. set[!, :PPFD] = set.Rs * 0.48 * 4.57
    set
end



,T,Wind,P,Rh,Ca,JMaxRef,VcMaxRef,RdRef,Rs
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,19.6128,7.31008,97.6876,0.652871,854.095,219.282,227.018,1.62698,338.431
2,30.4752,12.5113,92.1529,0.110913,528.443,211.471,204.605,1.35955,142.705
3,37.0693,14.7462,94.7316,0.466029,793.393,266.147,184.753,1.6229,106.442
4,36.1958,8.21692,94.8086,0.20033,392.997,296.88,177.223,1.36686,346.517
5,32.6337,13.8159,99.9659,0.806623,781.134,263.616,161.891,0.987039,306.333
6,39.8042,16.0392,90.4884,0.45714,750.567,283.968,157.291,1.71964,491.522
7,18.0286,15.1148,93.7844,0.863036,493.015,242.564,199.045,0.894889,453.445
8,36.3454,19.6451,91.8372,0.733399,496.796,205.461,186.324,1.36669,436.637
9,31.2409,9.08476,97.363,0.266161,436.04,240.004,182.193,0.820252,423.356



## Benchmarking




##### plantecophys

Preparing R to make the benchmark:

```julia
R\"\"\"
if(!require("plantecophys")){
    install.packages("plantecophys", repos = "https://cloud.r-project.org")
}
if(!require("microbenchmark")){
    install.packages("microbenchmark", repos = "https://cloud.r-project.org")
}
\"\"\"

# Make variables available to the R session
@rput set N microbenchmark_steps

```

Making the benchmark:

```julia
R\"\"\"
# Define the function call in a function that takes a list as input to limit DataFrame overhead
function_EB <- function(input) {
    PhotosynEB(
        Tair = input$Tair, VPD = input$VPD, Wind = input$Wind,
        Wleaf = input$Wleaf,Ca = input$Ca,  StomatalRatio = 1,
        LeafAbs = input$LeafAbs, gsmodel = "BBOpti", g0 = input$g0, g1 = input$g1,
        alpha = 0.24, theta = 0.7, Jmax = input$Jmax,
        Vcmax = input$Vcmax, TPU = input$TPU, Rd = input$Rd,
        RH = input$RH, PPFD=input$PPFD, Patm = input$Patm
    )
}

time_PE = c()
for(i in seq_len(N)){
    # Put the inputs into a vector to limit dataframe overhead:
    input = list(
        Tair = set$T[i], VPD = set$vpd[i], Wind = set$Wind[i], Wleaf = set$d[i],
        Ca = set$Ca[i], LeafAbs = set$sky_fraction[i], g0 = set$g0[i], g1 = set$g1[i],
        Jmax = set$JMaxRef[i], Vcmax = set$VcMaxRef[i], TPU = set$TPURef[i],
        Rd = set$RdRef[i], RH = set$Rh[i]*100, PPFD=set$PPFD[i],Patm = set$P[i]
    )

    m = microbenchmark(function_EB(input), times = microbenchmark_steps)

    time_PE = append(time_PE,m$time * 10e-9) # transform in seconds
}
\"\"\"

@rget time_PE
```






##### LeafGasExchange.jl

Note that we benchmark `LeafGasExchange.jl` with the `nounit` flag to compute a fair comparison with `PlantBiophysics.jl` in case computing units takes time (it shouldn't much).

```julia
time_LG = []
n_lg = fill(0, N)
for i = 1:N
    config =
        :Weather => (
            PFD=set.PPFD[i],
            CO2=set.Ca[i],
            RH=set.Rh[i] * 100,
            T_air=set.T[i],
            wind=set.Wind[i],
            P_air=set.P[i],
            g0=set.g0[i],
            g1=set.g1[i],
            Vcmax=set.VcMaxRef[i],
            Jmax=set.JMaxRef[i],
            Rd=set.RdRef[i],
            TPU=set.TPURef[i],
        )
    b_LG =
        @benchmark simulate($ModelC3MD; config=$config) evals = microbenchmark_evals samples =
            microbenchmark_steps
    append!(time_LG, b_LG.times .* 1e-9) # transform in seconds
    n_lg[i] = 1
end
```





##### PlantBiophysics.jl

Benchmarking `PlantBiophysics.jl`:

```julia
constants = Constants()
time_PB = []
for i = 1:N
    leaf = ModelList(
        energy_balance=Monteith(),
        photosynthesis=Fvcb(
            VcMaxRef=set.VcMaxRef[i],
            JMaxRef=set.JMaxRef[i],
            RdRef=set.RdRef[i],
            TPURef=set.TPURef[i],
        ),
        stomatal_conductance=Medlyn(set.g0[i], set.g1[i]),
        status=(
            Rₛ=set.Rs[i],
            sky_fraction=set.sky_fraction[i],
            PPFD=set.PPFD[i],
            d=set.d[i],
        ),
    )
    deps = PlantSimEngine.dep(leaf)
    meteo = Atmosphere(T=set.T[i], Wind=set.Wind[i], P=set.P[i], Rh=set.Rh[i], Cₐ=set.Ca[i])
    st = PlantMeteo.row_struct(leaf.status[1])
    b_PB = @benchmark run!($leaf, $deps, $st, $meteo, $constants, nothing) evals =
        microbenchmark_evals samples = microbenchmark_steps
    append!(time_PB, b_PB.times .* 1e-9) # transform in seconds
end
```




## Comparison




##### Statistics

We compute here basic statistics, _i.e._ mean, median, min, max, standard deviation. 

```julia
statsPB = basic_stat(time_PB)
statsPE = basic_stat(time_PE)
statsLG = basic_stat(time_LG)

factorPE = mean(time_PE) / mean(time_PB)
factorLG = mean(time_LG) / mean(time_PB)

# Write overall timings:
df = DataFrame(
	[getfield(j, i) for i in fieldnames(StatResults), j in [statsPB, statsPE, statsLG]],
	["PlantBiophysics", "plantecophys", "LeafGasExchange"]
)
insertcols!(df, 1, :Stat => [fieldnames(StatResults)...])
CSV.write("benchmark.csv", df)

# Write timing for each sample:
CSV.write("benchmark_full.csv",
	DataFrame(
		"package" => vcat(
			[
				repeat([i.first], length(i.second)) for i in [
					"PlantBiophysics" => time_PB,
					"plantecophys" => time_PE,
					"LeafGasExchange" => time_LG
				]
			]...
		),
		"sample_time" => vcat(time_PB, time_PE, time_LG)
	)
)
```



In [7]:
df_res = CSV.read(Downloads.download("https://raw.githubusercontent.com/VEZY/PlantBiophysics-paper/main/notebooks/performance/benchmark.csv"), DataFrame)



,Stat,PlantBiophysics,plantecophys,LeafGasExchange
,String7,Float64,Float64,Float64
1,mean,1.21661e-6,0.0470207,0.0226825
2,median,1.083e-6,0.04569,0.0214673
3,stddev,1.20394e-6,0.00808695,0.0047238
4,min,5.41e-7,0.0346401,0.0192761
5,max,2.5e-5,0.172202,0.12373



##### Histogram plotting



In [8]:
"""
	StatResults(
		mean::AbstractFloat
    	median::AbstractFloat
    	stddev::AbstractFloat
    	min::AbstractFloat
    	max::AbstractFloat
	)

Structure to hold basic statistics of model performance.
"""
struct StatResults
    mean::AbstractFloat
    median::AbstractFloat
    stddev::AbstractFloat
    min::AbstractFloat
    max::AbstractFloat
end



StatResults


    **Base.show**

    	Base.show(io::IO, m::StatResults)
    	Base.show(io::IO, ::MIME"text/plain", m::StatResults)

    Add a show method for our `StatResults` type.
    


In [9]:
"""
	basic_stat(df)

Compute basic statistics from the benchmarking
"""
function basic_stat(df)
    m = mean(df)
    med = median(df)
    std = Statistics.std(df)
    min = findmin(df)[1]
    max = findmax(df)[1]
    return StatResults(m, med, std, min, max)
end



basic_stat

In [10]:
function plot_benchmark_Makie(statsPB, statsPE, statsLG, time_PB, time_PE, time_LG)
    size_inches = (6.7, 5)
    size_pt = 72 .* size_inches
    bins = 220
    noto_sans = assetpath("fonts", "NotoSans-Regular.ttf")
    fig = Figure(
        backgroundcolor=RGBf(1, 1, 1),
        resolution=size_pt,
        font=noto_sans,
        fontsize=10,
    )
    ep = 1e-9
    extr = extrema(vcat(time_PB, time_PE, time_LG))
    interval = (extr[1] * 1e-1, extr[2])

    ga = fig[1, 1] = GridLayout()

    axa = Axis(
        ga[1, 1],
        title="(a) PlantBiophysics.jl",
        xscale=log10,
        titlealign=:left,
        titlesize=10,
    )
    stddevi = poly!(
        axa,
        Rect(max(ep, statsPB.mean - statsPB.stddev), 0.0, 2 * statsPB.stddev, 1),
        color=(:orange, 0.3),
        yautolimits=false,
    )
    moy = vlines!(axa, statsPB.mean; color=:red, linewidth=3, linestyle=:dot)
    hist!(axa, time_PB, normalization=:probability, bins=bins)
    # h = axa.finallimits[].widths[2]
    axislegend(
        axa,
        [stddevi, moy],
        ["95% confidence interval", "Mean"],
        "",
        position=:rb,
        orientation=:vertical,
        labelsize=8,
        framevisible=false,
    )
    xlims!(axa, interval)

    axb = Axis(
        ga[2, 1],
        title="(b) plantecophys",
        xscale=log10,
        ylabel="Density",
        titlealign=:left,
        titlesize=10,
    )
    stddevi = poly!(
        axb,
        Rect(statsPE.mean - statsPE.stddev, 0.0, 2 * statsPE.stddev, 1),
        color=(:orange, 0.3),
        yautolimits=false,
    )
    vlines!(axb, statsPE.mean; color=:red, linewidth=3, linestyle=:dot)
    hist!(axb, time_PE, normalization=:probability, bins=bins)
    xlims!(axb, interval)

    # axc = Axis(ga[3, 1], title="(c) LeafGasExchange.jl", yminorticks=IntervalsBetween(10),
    #     xscale=log10, xminorticks=IntervalsBetween(10), yminorgridvisible=true, yminorticksvisible=true,
    #     xminorgridvisible=true, xminorticksvisible=true, xlabel="Time (s)")
    axc = Axis(
        ga[3, 1],
        title="(c) LeafGasExchange.jl",
        xscale=log10,
        xlabel="Time (s)",
        titlealign=:left,
        titlesize=10,
    )
    stddevi = poly!(
        axc,
        Rect(statsLG.mean - statsLG.stddev, 0.0, 2 * statsLG.stddev, 1),
        color=(:orange, 0.3),
        yautolimits=false,
    )
    vlines!(axc, statsLG.mean; color=:red, linewidth=3, linestyle=:dot)
    hist!(axc, time_LG, normalization=:probability, bins=bins)
    xlims!(axc, interval)

    rowgap!(ga, 7)
    hidexdecorations!(axa, grid=false)
    hidexdecorations!(axb, grid=false)
    fig
end



plot_benchmark_Makie (generic function with 1 method)

We here display the computational time histogram of each package on the same scale in order to compare them: `PlantBiophysics.jl` (a), `plantecophys` (b) and `LeafGasExchange.jl` (c). The y-axis represents the density (_i.e._ reaching 0.3 means that 30% of the computed times are in this bar). Orange zone represents the interval [mean - standard deviation; mean + standard deviation]. Red dashed line represents the mean. Note that the x-axis is logarithmic.


```{julia}
fig = plot_benchmark_Makie(statsPB, statsPE, statsLG, time_PB, time_PE, time_LG)
save("benchmark_each_time_steps.png", fig, px_per_unit=3)
``````


![](https://raw.githubusercontent.com/VEZY/PlantBiophysics-paper/main/notebooks/performance/benchmark_each_time_steps.png)

> :memo: **Note** <br>
PlantBiophysics.jl is $(Int(round(df_res[1,:plantecophys] / df_res[1,:PlantBiophysics]))) times faster than plantecophys, and  $(Int(round(df_res[1,:LeafGasExchange] / df_res[1,:PlantBiophysics]))) times faster than LeafGasExchange.jl.

> :warning: **Warning** <br>
This is the plot from the latest commit on <https://github.com/VEZY/PlantBiophysics-paper/>. If you want to make your own benchmarking, run the script that was used to perform it, but careful, it takes a long time to perform!

